# Part2: Model Training

<h3>2.1 Baseline Model </h3>

We first make a random forset model as a baseline model

In [1]:
import pandas as pd
import numpy as np
import os
import logging
from data import DATA_DIR
from lama.preprocessing.DataProcessor import nans, reformat_dataframe

logger = logging.getLogger()
PRE_DIR = os.path.join(DATA_DIR, "pre")

We do not need card id anynore since we don't have to merge tables

In [5]:
train = pd.read_csv(os.path.join(PRE_DIR, "train_groupby.csv"))
test = pd.read_csv(os.path.join(PRE_DIR, "test_groupby.csv"))

features = train.columns.tolist()
features.remove('target')
features.remove('card_id')


ValueError: list.remove(x): x not in list

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

In [ ]:
PARAMETER_SPACE = {
    "n_estimators": range(100),
    "min_samples_leaf": range(50),
    "min_samples_split": range(10),
    "max_depth": range(20),
    "max_features": ["auto", 80]
}

In [ ]:
clf = RandomForestRegressor(criterion='squared_error', n_jobs=16, random_state=42)
grid = GridSearchCV(clf, PARAMETER_SPACE, cv=2, scoring='neg_mean_squared_error')
print(train['target'].values)
grid.fit(train[features].values, train['target'].values)

[-0.8202826   0.39291325  0.68805599 ...  0.09349415 -4.67658938
 -1.85941301]


/opt/homebrew/Caskroom/miniforge/base/envs/lama_gpu/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
224 fits failed out of a total of 800.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
80 fits failed with the following error:
Traceback (most recent call last):
  File "/opt/homebrew/Caskroom/miniforge/base/envs/lama_gpu/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 681, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/opt/homebrew/Caskroom/miniforge/base/envs/lama_gpu/lib/python3.9/site-packages/sklearn/ensemble/_forest.py", line 384, in fit
    self._validate_estimator()
  File "/opt/homebrew/Caskroom/miniforge/base/envs/lama_gpu/lib/pyth

GridSearchCV(cv=2, estimator=RandomForestRegressor(n_jobs=15, random_state=42),
             param_grid={'max_depth': [9, 10], 'max_features': ['auto', 80],
                         'min_samples_leaf': range(0, 5),
                         'min_samples_split': [2, 3],
                         'n_estimators': range(0, 10)},
             scoring='neg_mean_squared_error')

In [ ]:
grid.best_params_

{'max_depth': 9,
 'max_features': 80,
 'min_samples_leaf': 4,
 'min_samples_split': 2,
 'n_estimators': 9}

In [ ]:
best_score = np.sqrt(-grid.best_score_)
_result = \
{
    "model": "RandomForestRegressor",
    "best_params": grid.best_params_,
    "best_score": best_score,
}


In [ ]:
test['target'] = grid.best_estimator_.predict(test[features])

/opt/homebrew/Caskroom/miniforge/base/envs/lama_gpu/lib/python3.9/site-packages/sklearn/base.py:438: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(


In [ ]:
import json

RESULT_DIR = os.path.join(DATA_DIR, "result")
test[['card_id', 'target']].to_csv(os.path.join(RESULT_DIR, "submission_random_forest_1.csv"), index=False)
with open(os.path.join(RESULT_DIR, "hyperparameters"), "a") as file:
    file.write(json.dumps(_result) + "\n")

After submit our first baseline model into kagggle, we get the result.

<img src="../../assets/img/Screenshot_1.png" />

We can see that this result is not very performing. So we will try out other model or make hyperparameter optimizations based on this result.

<h3>2.2 Baseline Model </h3>

Now we try to build the model the second time, by implementing PCA and other filter model, we will run our random forest model a second time, hoping it achieve a better performance